In [1]:
import Pkg
#Pkg.add("MathOptInterface")
#Pkg.add("GLPK")
Pkg.activate(@__DIR__)
Pkg.instantiate()

Activating environment at `C:\Users\mmart\Desktop\TFM\Julia\notebooks\Project.toml`


In [2]:
using JuMP
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

In [3]:
include("../src/read_data.jl")
include("../JuMP/main_JuMP.jl")

H0 = 20

Constants(5, 5, 5, 2, 490, 1, 1)

In [4]:
T=P+1
PP=[p for p in 1:P]
PPP=[p for p in 0:T]
PP0=[p for p in 0:P]
PPT=[p for p in 1:T]
QQ=[q for q in 1:Q]
CC=[c for c in 1:C]

#create bj_dict
bj_dict = Dict{Int, Array{Int,1}}()
bj_init = [1,4]
for p in PPP
    l = Array{Int,1}()
    for q in QQ
        if p == 0
            push!(l, bj_init[q])
        elseif p == T
            push!(l, 0)
        else
            push!(l, bj[p])
        end
    end
    bj_dict[p] = l
end

function travel_time(current_bay::Int, target_bay::Int, CTS::Constants)
    if current_bay == 0 || target_bay == 0
        return(0)
    else
        return(abs(current_bay - target_bay)*CTS.tt)
    end
end

function clearance_travel_time(i::Int, j::Int, v::Int, w::Int, bj_dict::Dict{Int, Array{Int,1}}, CTS::Constants)
    delta = (CTS.delta + 1)*abs(v-w)
    li = bj_dict[i][v]
    lj = bj_dict[j][w]
    if (v < w) && (i != j) && (li > lj - delta)
        return((li - lj + delta)*CTS.tt)
    elseif (v > w) && (i != j) && (li < lj + delta)
        return((lj - li + delta)*CTS.tt)
    else
        return(0)
    end
end

clearance_travel_time (generic function with 1 method)

In [5]:
clearance_travel_time(1,2,1,2,bj_dict,CTS)

2

In [6]:
model = Model(optimizer_with_attributes(GLPK.Optimizer));
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

In [7]:
#if positions p, s are performed consecutively by crane q
@variable(model, x[p=0:T, s=0:T, q=1:Q], Bin)
#if positions p is filled with container i
@variable(model, w[p=0:T, i=1:C], Bin)
#if position s starts after the completion time of task p
@variable(model, z[p=1:P, s=1:P], Bin)

#completion time of task p
@variable(model, t_load[p=0:T], Int, lower_bound=0, upper_bound=CTS.H)
#completion required time to perform task p
@variable(model, t_task[p=0:T], lower_bound=0, upper_bound=CTS.H)
#completion time of crane q
#@variable(model, t_crane[q=1:Q], lower_bound=0, upper_bound=CTS.H)
#makespan
#@variable(model, makespan, lower_bound=0, upper_bound=CTS.H)


#objective function
#@objective(model, Min, 0.8*t_load[T] + 0.2*sum(t_crane[q] for q in QQ))
@objective(model, Min, t_load[T])

t_load[6]

In [8]:
@constraint(model, [p=0:T, q=1:Q], x[p,p,q] == 0)
@constraint(model, [p=1:P], z[p,p] == 0)

@constraint(model, t_load[0] == 0)
@constraint(model, t_load[T] <= CTS.H)

t_load[6] <= 490.0

In [9]:
@constraint(model, t_task[0] == 0)
@constraint(model, t_task[T] == 0)

@constraint(model, [p=1:P], t_task[p] == sum(task_times[p,i]*w[p,i] for i in subset_pos(CC, tasks_by_position, p)))

5-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 t_task[1] - 22 w[1,1] - 60 w[1,2] - 5 w[1,4] == 0.0 
 t_task[2] - 38 w[2,1] - 46 w[2,2] - 40 w[2,4] == 0.0
 t_task[3] - 8 w[3,3] == 0.0                         
 t_task[4] - 16 w[4,1] - 62 w[4,2] - 70 w[4,4] == 0.0
 t_task[5] - 10 w[5,5] == 0.0                        

In [10]:
@constraint(model, [i=1:C], w[0,i] == 0)
@constraint(model, [i=1:C], w[T,i] == 0)

@constraint(model, [i=1:C], sum(w[p,i] for p in subset_pos(PP, tasks_by_position, i)) == 1)
@constraint(model, [p=1:P], sum(w[p,i] for i in subset_pos(CC, tasks_by_position, p)) == 1)

5-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 w[1,1] + w[1,2] + w[1,4] == 1.0
 w[2,1] + w[2,2] + w[2,4] == 1.0
 w[3,3] == 1.0                  
 w[4,1] + w[4,2] + w[4,4] == 1.0
 w[5,5] == 1.0                  

In [11]:
#@constraint(model, [q=1:Q], t_crane[q] <= t_load[T])

In [12]:
@constraint(model, [q=1:Q], sum(x[0,s,q] for s in PPT) == 1)
@constraint(model, [q=1:Q], sum(x[p,T,q] for p in PP0) == 1)

2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 x[0,6,1] + x[1,6,1] + x[2,6,1] + x[3,6,1] + x[4,6,1] + x[5,6,1] == 1.0
 x[0,6,2] + x[1,6,2] + x[2,6,2] + x[3,6,2] + x[4,6,2] + x[5,6,2] == 1.0

In [13]:
@constraint(model, [p=1:P], sum(sum(x[p,s,q] for s in PPT) for q in QQ) == 1)

@constraint(model, [p=1:P, q=1:Q], sum(x[s,p,q] for s in PP0) - sum(x[p,s,q] for s in PPT) == 0)

5×2 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},2}:
 x[0,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] - x[1,2,1] - x[1,3,1] - x[1,4,1] - x[1,5,1] - x[1,6,1] == 0.0  …  x[0,1,2] + x[2,1,2] + x[3,1,2] + x[4,1,2] + x[5,1,2] - x[1,2,2] - x[1,3,2] - x[1,4,2] - x[1,5,2] - x[1,6,2] == 0.0
 x[0,2,1] + x[1,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] - x[2,1,1] - x[2,3,1] - x[2,4,1] - x[2,5,1] - x[2,6,1] == 0.0     x[0,2,2] + x[1,2,2] + x[3,2,2] + x[4,2,2] + x[5,2,2] - x[2,1,2] - x[2,3,2] - x[2,4,2] - x[2,5,2] - x[2,6,2] == 0.0
 x[0,3,1] + x[1,3,1] + x[2,3,1] + x[4,3,1] + x[5,3,1] - x[3,1,1] - x[3,2,1] - x[3,4,1] - x[3,5,1] - x[3,6,1] == 0.0     x[0,3,2] + x[1,3,2] + x[2,3,2] + x[4,3,2] + x[5,3,2] - x[3,1,2] - x[3,2,2] - x[3,4,2] - x[3,5,2] - x[3,6,2] == 0.0
 x[0,4,1] + x[1,4,1] + x[2,4,1] + x[3,4,1] + x[5,4,1] - x[4,1,1] - x[4,2,1] - x[4,3,1] - x[4,5,1] - x[4,6,1] == 0.0     x[0,4,2] + x[

In [14]:
for s in PP
    for p in prec[s]
        @constraint(model, t_load[p] + 2*t_task[s] - t_load[s] <= 0)
        @constraint(model, z[p,s] + z[s,p] == 1)
    end
end

In [15]:
for p in PPP
    for s in PPP
        if p != s
            for q in QQ
                @constraint(model, t_load[p] + travel_time(bj_dict[p][q], bj_dict[s][q], CTS) + 2*t_task[s] - t_load[s] <= CTS.H*(1-x[p,s,q]))
            end
        end
    end
end

for p in PP
    for s in PP
        if p != s
            @constraint(model, t_load[p] + 2*t_task[s] - t_load[s] <= CTS.H*(1-z[p,s]))
            @constraint(model, t_load[s] - 2*t_task[s] - t_load[p] <= CTS.H*z[p,s])

            #if bj_dict[p] == bj_dict[s]
            #    custom_set=Array{Int,1}()
            #    custom_set = collect(setdiff(Set(PP0),Set(bay_to_pos(bj_dict[p], bj))))
            #    @constraint(model, t_load[p] + 2*t_task[s] - t_load[s] + CTS.tt*sum(sum(x[u,s,q] for u in custom_set) for q in QQ) <= CTS.H*(1-z[p,s]))
            #    @constraint(model, t_load[s] - 2*t_task[s] - t_load[p] + CTS.tt*sum(sum(x[u,s,q] for u in custom_set) for q in QQ) <= CTS.H*z[p,s])
            #end
        end
    end
end

In [16]:
#@constraint(model, [p=1:P, q=1:Q], t_load[p] + travel_time(p, bj_start_end[1, q], CTS) - t_crane[q] <= CTS.H*(1-x[p,T,q]))
#@constraint(model, [p=1:P, q=1:Q], t_load[p] - t_crane[q] <= CTS.H*(1-x[p,T,q]))

@constraint(model, [s=1:P, q=1:Q], travel_time(bj_dict[0][q], bj_dict[s][q], CTS) + 2*t_task[s] - t_load[s] <= CTS.H*(1-x[0,s,q]))
#@constraint(model, [s=1:P, q=1:Q], t_task[s] - t_load[s] <= CTS.H*(1-x[0,s,q]))

5×2 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},2}:
 2 t_task[1] - t_load[1] + 490 x[0,1,1] <= 490.0  …  2 t_task[1] - t_load[1] + 490 x[0,1,2] <= 487.0
 2 t_task[2] - t_load[2] + 490 x[0,2,1] <= 490.0     2 t_task[2] - t_load[2] + 490 x[0,2,2] <= 487.0
 2 t_task[3] - t_load[3] + 490 x[0,3,1] <= 489.0     2 t_task[3] - t_load[3] + 490 x[0,3,2] <= 488.0
 2 t_task[4] - t_load[4] + 490 x[0,4,1] <= 488.0     2 t_task[4] - t_load[4] + 490 x[0,4,2] <= 489.0
 2 t_task[5] - t_load[5] + 490 x[0,5,1] <= 486.0     2 t_task[5] - t_load[5] + 490 x[0,5,2] <= 489.0

In [17]:
for p in PP
    for s in PP
        for v in QQ
            for w in QQ
                if p < s && clearance_travel_time(p, s, v, w, bj_dict, CTS) > 0
                    @constraint(model, sum(x[u,p,v] for u in PP0) + sum(x[u,s,w] for u in PP0) <= 1 + z[p,s] + z[s,p])
                    @constraint(model, t_load[p] + clearance_travel_time(p, s, v, w, bj_dict, CTS) + 2*t_task[s] - t_load[s] <= CTS.H*(3 - z[p,s] - sum(x[u,p,v] for u in PP0) - sum(x[u,s,w] for u in PP0)))
                    @constraint(model, t_load[s] + clearance_travel_time(p, s, v, w, bj_dict, CTS) + 2*t_task[p] - t_load[p] <= CTS.H*(3 - z[s,p] - sum(x[u,p,v] for u in PP0) - sum(x[u,s,w] for u in PP0)))
                end
            end
        end
    end
end

## Solution

In [18]:
JuMP.optimize!(model) # Old syntax: status = JuMP.solve(model)

In [19]:
makespan = JuMP.value.(t_load)[T]

119.0

In [20]:
sol_x = Dict{Int, Array}()
for q=1:Q
    for p=0:T
        for s=0:T
            if JuMP.value.(x)[p,s,q] == 1
                if haskey(sol_x, q) == false
                    sol_x[q] = Array{NamedTuple{(:start_time, :pos, :next_pos, :qc),Tuple{Int64,Int64,Int64,Int64}}, 1}()
                end
                push!(sol_x[q], (start_time=JuMP.value.(t_load)[p]-2*JuMP.value.(t_task)[p], pos=p, next_pos=s, qc=q))
            end                
        end
    end
    sort!(sol_x[q], by = x->x.start_time)
end

sol_x[1]

4-element Array{NamedTuple{(:start_time, :pos, :next_pos, :qc),NTuple{4,Int64}},1}:
 (start_time = 0, pos = 0, next_pos = 1, qc = 1)  
 (start_time = 0, pos = 1, next_pos = 2, qc = 1)  
 (start_time = 10, pos = 2, next_pos = 3, qc = 1) 
 (start_time = 103, pos = 3, next_pos = 6, qc = 1)

In [21]:
sol_x[2]

3-element Array{NamedTuple{(:start_time, :pos, :next_pos, :qc),NTuple{4,Int64}},1}:
 (start_time = 0, pos = 0, next_pos = 4, qc = 2) 
 (start_time = 10, pos = 4, next_pos = 5, qc = 2)
 (start_time = 83, pos = 5, next_pos = 6, qc = 2)

In [22]:
sol_w = Dict{Int, Int}()
for p=1:P
    for i=1:C
        if JuMP.value.(w)[p,i] == 1
            sol_w[p] = i             
        end
    end
end
sol_w[0] = 0
sol_w[T] = 0

sol_w

Dict{Int64,Int64} with 7 entries:
  0 => 0
  4 => 1
  2 => 2
  3 => 3
  5 => 5
  6 => 0
  1 => 4

In [23]:
LS = init_ls(CTS)
QC = init_qc(CTS)

2-element Array{QuayCrane,1}:
 QuayCrane(1, [1, 2, 3], "idle", 0, 1, 1, Task[])
 QuayCrane(2, [3, 4, 5], "idle", 0, 3, 3, Task[])

In [24]:
function update_ls(LS::LoadingSequence, t::NamedTuple{(:start_time, :pos, :next_pos, :qc),Tuple{Int64,Int64,Int64,Int64}}, sol_w::Dict{Int, Int}, CTS::Constants)
    if t.pos != 0
        LS.len += 1
        LS.tasks_left -= 1
        push!(LS.filled_pos, t.pos)
        push!(LS.loaded_cont, sol_w[t.pos])
    end
    push!(LS.order, (task=Task(t.pos, bj_dict[t.pos][t.qc], sol_w[t.pos], JuMP.value.(t_task)[t.pos]), start_time=t.start_time, qc=t.qc))
end

update_ls (generic function with 1 method)

In [25]:
while LS.tasks_left > 0
    task = [(start_time=CTS.H+1,pos=0,next_pos=0,qc=0)]
    for q=1:Q
        if length(sol_x[q])>0 && sol_x[q][1].start_time < task[1].start_time
            task = [sol_x[q][1]]
        elseif length(sol_x[q])>0 && sol_x[q][1].start_time == task[1].start_time
            push!(task, sol_x[q][1])
        end
    end

    for t in task
        update_ls(LS, t, sol_w, CTS)
        if sol_x[t.qc][1] == t
            deleteat!(sol_x[t.qc], 1)
        end
    end
end

In [26]:
LS

LoadingSequence(NamedTuple{(:task, :start_time, :qc),Tuple{Task,Int64,Int64}}[(task = Task(0, 1, 0, 0), start_time = 0, qc = 1), (task = Task(0, 4, 0, 0), start_time = 0, qc = 2), (task = Task(1, 1, 4, 5), start_time = 0, qc = 1), (task = Task(2, 1, 2, 46), start_time = 10, qc = 1), (task = Task(4, 3, 1, 16), start_time = 10, qc = 2), (task = Task(5, 5, 5, 10), start_time = 83, qc = 2), (task = Task(3, 2, 3, 8), start_time = 103, qc = 1)], 0, 5, [1, 2, 4, 5, 3], [4, 2, 1, 5, 3])

In [27]:
function check_tasks(LS::LoadingSequence, CTS::Constants)
    if length(LS.filled_pos) == P && length(LS.loaded_cont) == C && length(setdiff(LS.filled_pos, collect(1:P))) == 0 && length(setdiff(LS.loaded_cont, collect(1:C))) == 0
        return(true)
    else
        return(false)
    end
end

function check_clearance(QC::Array{QuayCrane, 1}, q::Int, target_bay::Int, CTS::Constants)
    if QC[q-1].current_bay < target_bay - CTS.delta && QC[q+1].current_bay > target_bay + CTS.delta
        return(true)
    end
    return(false)
end

function check_prec(LS::LoadingSequence, task::Task, precedences::Dict)
    for req in precedences[task.p]
        if !(req in LS.filled_pos)
            print("trying to add position, but still missing")
            print(setdiff(precedences[task.p],LS.filled_pos))
            return(false)
        end
    end
    return(true)
end


check_prec (generic function with 1 method)

In [28]:
check_tasks(LS,CTS)

true

In [29]:
for task in LS.order
    if task[1].p != 0
        if check_prec(LS,task[1],prec) == false
            return(false)
        end
    end
end
return(true)

true

In [30]:
#get QC moves and times
function get_qc_moves(LS::LoadingSequence, CTS::Constants)
    QC_MOVES=Dict{Int, Array{NamedTuple{(:bay, :start_time, :time),Tuple{Int64,Int64,Int64}}, 1}}()
    for q = 1:CTS.Q
        l=Array{NamedTuple{(:bay, :start_time, :time),Tuple{Int64,Int64,Int64}}, 1}()
        for t in LS.order
            if t.qc == q
                push!(l, (bay=t.task.b, start_time=t.start_time, time=t.task.t))
            end
        end
        QC_MOVES[q]=l
    end
    return(QC_MOVES)
end

function check_solution(LS::LoadingSequence, QC::Array{QuayCrane, 1}, CTS::Constants)
    #check positions and containers
    if check_tasks(LS, CTS) == false
        return(false)
    end

    #check the QC constraints
    QC_MOVES=get_qc_moves(LS, CTS)

    for clock in 1:CTS.H
        current_cranes=[]
        for (key, value) in QC_MOVES
            if clock == 1
                for i = 1: length(value)-1
                    #check loading + travel time
                    if value[i].start_time + 2*value[i].time + travel_time(value[i].bay, value[i+1].bay, CTS) != value[i+1].start_time
                        print("start_time_i = ")
                        print(value[i].start_time)
                        print("    task_time_i = ")
                        print(value[i].time)
                        print("    travel = ")
                        print(travel_time(value[i].bay, value[i+1].bay, CTS))
                        print("    start_time_i+1 = ")
                        print(value[i+1].start_time)
                        print("    bay_i = ")
                        print(value[i].bay)
                        print("    bay_i+1 = ")
                        print(value[i+1].bay)
                        print("\n")
                        #return(false)
                    end
                    #check that cranes respect bays
                    if !(value[i].bay in QC[key].available_bays)
                        print("wrong bays")
                        return(false)
                    end
                end
                if !(value[length(value)].bay in QC[key].available_bays)
                    print("wrong bays")
                    return(false)
                end
            end

            #s'hauria de millorar per si una grua te idle time
            for tuple in value
                if tuple.start_time <= clock && clock <= tuple.start_time + tuple.time
                    push!(current_cranes, (key, tuple.bay))
                end
            end
        end

        #check clearance
        sort(current_cranes, by = first)
        for b = 1:length(current_cranes)-1
            if current_cranes[b] - current_cranes[b+1] < CTS.delta
                print("clearance")
                return(false)
            end
        end
    end
end


check_solution (generic function with 1 method)

In [31]:
check_solution(LS, QC, CTS)

start_time_i = 0    task_time_i = 0    travel = 1    start_time_i+1 = 10    bay_i = 4    bay_i+1 = 3
start_time_i = 10    task_time_i = 16    travel = 2    start_time_i+1 = 83    bay_i = 3    bay_i+1 = 5


MethodError: MethodError: no method matching -(::Tuple{Int64,Int64}, ::Tuple{Int64,Int64})
Closest candidates are:
  -(!Matched::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}, ::T) where T at C:\Users\mmart\.julia\packages\MathOptInterface\XiH8D\src\Utilities\functions.jl:1125
  -(::T, !Matched::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}) where T at C:\Users\mmart\.julia\packages\MathOptInterface\XiH8D\src\Utilities\functions.jl:1131